In [29]:
import json
from transformers import BertTokenizer
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer
import torch.nn as nn
import torch.utils as utils
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

In [2]:
f=open("test.json","r",encoding="utf-8")

In [3]:
data=json.load(f)

In [4]:
data[0]

{'id': '190315_E001_13',
 'tag': 'meeting',
 'title': 'Meeting: Market update meeting',
 'original_language': 'en',
 'conversation': [{'no': 1,
   'en_speaker': 'Mr. John Smith',
   'ja_speaker': 'ジョン スミスさん',
   'en_sentence': 'How is it going, Wayne?',
   'ja_sentence': 'ウェイン、調子はどうです？'},
  {'no': 2,
   'en_speaker': 'Mr. Wayne Willis',
   'ja_speaker': 'ウェイン ウィリスさん',
   'en_sentence': "I'm not too bad.",
   'ja_sentence': 'まあまあです。'},
  {'no': 3,
   'en_speaker': 'Mr. John Smith',
   'ja_speaker': 'ジョン スミスさん',
   'en_sentence': 'Thank you very much for coming out today.',
   'ja_sentence': '今日はご足労ありがとう。'},
  {'no': 4,
   'en_speaker': 'Mr. John Smith',
   'ja_speaker': 'ジョン スミスさん',
   'en_sentence': "How's business lately?",
   'ja_sentence': '景気はどうです？'},
  {'no': 5,
   'en_speaker': 'Mr. Wayne Willis',
   'ja_speaker': 'ウェイン ウィリスさん',
   'en_sentence': "It's been good.",
   'ja_sentence': 'おかげさまで、順調です。'},
  {'no': 6,
   'en_speaker': 'Mr. Wayne Willis',
   'ja_speaker': 'ウェイン ウィリスさん',


In [5]:
english=[]
japanese=[]

In [6]:
for dat in data:
    for j in dat['conversation']:
        english.append(j['en_sentence'])
        japanese.append(j['ja_sentence'])

In [7]:
len(english)

2120

In [8]:
len(japanese)

2120

In [9]:
mx_eng=0
mx_jp=0
for i in range(len(english)):
    mx_eng=max(mx_eng,len(english[i]))
for i in range(len(japanese)):
    mx_jp=max(mx_jp,len(japanese[i]))    

Max of english and Max of Japanese tokens 

In [10]:
print(mx_eng,mx_jp)

207 95


In [11]:
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer

In [12]:
bertjapanese = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")


C:\Users\maddo\anaconda3\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [13]:
special_tokens = {
    "bos_token": "<sos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>"
}

In [14]:
tokenizer.add_special_tokens(special_tokens)

3

In [15]:
jptokids=[]
jptorch=[]
for i in range(len(japanese)):
    ax = tokenizer.tokenize(japanese[i])
    ax = [tokenizer.bos_token] + ax + [tokenizer.eos_token]  
    jptokids.append(tokenizer.convert_tokens_to_ids(ax))
    jptorch.append(torch.tensor(jptokids[-1]))

This is using tokenizer.encode to automatically add eos and sos 

for i in range(len(japanese)):
    jptokids.append(tokenizer.encode(japanese[i], add_special_tokens=True))  
    print(jptokids[-1])

In [16]:
tokenizer.pad_token_id

32002

In [17]:
maxlen_jp=0
vocabsize_jp=0
for i in range(len(jptokids)):
    maxlen_jp=max(maxlen_jp,len(jptokids[i]))
    vocabsize_jp = max(vocabsize_jp,max(jptokids[i]))
    

In [18]:
maxlen_jp,vocabsize_jp

(59, 32001)

In [19]:
engtokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [20]:
engtokenizer.add_special_tokens(special_tokens)

3

In [21]:
engtokids=[]
engtorch=[]
for i in range(len(english)):
    ax=engtokenizer.tokenize(english[i])
    ax=[engtokenizer.bos_token]+ax+[engtokenizer.eos_token]
    engtokids.append(engtokenizer.convert_tokens_to_ids(ax))
    engtorch.append(torch.tensor(engtokids[-1]))

In [22]:
maxlen_eng=0
vocabsize_eng=0
for i in range(len(engtokids)):
    maxlen_eng=max(maxlen_eng,len(engtokids[i]))
    vocabsize_eng = max(vocabsize_eng,max(engtokids[i]))
    

In [23]:
maxlen_eng,vocabsize_eng,maxlen_jp,vocabsize_jp

(45, 30523, 59, 32001)

In [24]:
tokenizer.decode(jptokids[0], skip_special_tokens=False)

'<sos> ウェイン 、 調子 は どう です ? <eos>'

In [25]:
engtokenizer.decode(engtokids[0], skip_special_tokens=True)

'how is it going, wayne?'

In [26]:
engpadded= pad_sequence(engtorch, batch_first=True, padding_value=engtokenizer.pad_token_id)

In [27]:
jppadded= pad_sequence(jptorch,batch_first=True ,padding_value=tokenizer.pad_token_id)

In [31]:
class TranslationDataset(Dataset):
    def __init__(self, en_tensors, jp_tensors):
        self.en_tensors = en_tensors
        self.jp_tensors = jp_tensors

    def __len__(self):
        return len(self.en_tensors)

    def __getitem__(self, idx):
        # Return English and Japanese tensors for each sentence pair
        return self.en_tensors[idx], self.jp_tensors[idx]

In [32]:
def get_dataloader(en_tensors, jp_tensors, batch_size=32):
    dataset = TranslationDataset(en_tensors, jp_tensors)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [33]:
train_loader = get_dataloader(engpadded, jppadded, batch_size=32)

In [46]:
for i in next(iter(train_loader)):
    print(i)

tensor([[30522,  2011,  1996,  ..., 30524, 30524, 30524],
        [30522,  2065,  2057,  ..., 30524, 30524, 30524],
        [30522,  2748,  1010,  ..., 30524, 30524, 30524],
        ...,
        [30522,  1045,  2064,  ..., 30524, 30524, 30524],
        [30522,  2031,  2017,  ..., 30524, 30524, 30524],
        [30522,  2256,  3579,  ..., 30524, 30524, 30524]])
tensor([[32000,    22,     6,  ..., 32002, 32002, 32002],
        [32000,  6045,  5826,  ..., 32002, 32002, 32002],
        [32000,    22, 28474,  ..., 32002, 32002, 32002],
        ...,
        [32000,    83,  6867,  ..., 32002, 32002, 32002],
        [32000,  8277,    35,  ..., 32002, 32002, 32002],
        [32000, 25559, 28609,  ..., 32002, 32002, 32002]])


In [61]:
for i,j in enumerate(train_loader):
    print(i,j[0].shape)

0 torch.Size([32, 45])
1 torch.Size([32, 45])
2 torch.Size([32, 45])
3 torch.Size([32, 45])
4 torch.Size([32, 45])
5 torch.Size([32, 45])
6 torch.Size([32, 45])
7 torch.Size([32, 45])
8 torch.Size([32, 45])
9 torch.Size([32, 45])
10 torch.Size([32, 45])
11 torch.Size([32, 45])
12 torch.Size([32, 45])
13 torch.Size([32, 45])
14 torch.Size([32, 45])
15 torch.Size([32, 45])
16 torch.Size([32, 45])
17 torch.Size([32, 45])
18 torch.Size([32, 45])
19 torch.Size([32, 45])
20 torch.Size([32, 45])
21 torch.Size([32, 45])
22 torch.Size([32, 45])
23 torch.Size([32, 45])
24 torch.Size([32, 45])
25 torch.Size([32, 45])
26 torch.Size([32, 45])
27 torch.Size([32, 45])
28 torch.Size([32, 45])
29 torch.Size([32, 45])
30 torch.Size([32, 45])
31 torch.Size([32, 45])
32 torch.Size([32, 45])
33 torch.Size([32, 45])
34 torch.Size([32, 45])
35 torch.Size([32, 45])
36 torch.Size([32, 45])
37 torch.Size([32, 45])
38 torch.Size([32, 45])
39 torch.Size([32, 45])
40 torch.Size([32, 45])
41 torch.Size([32, 45])
42